In [1]:
import sys
sys.path.append('..')
from mxnet import autograd, nd
from mxnet.gluon import loss as gloss
import random
import zipfile
import utils

/usr/local/lib/python3.5/dist-packages/scipy/_lib/decorator.py:205: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  first = inspect.getargspec(caller)[0][0]  # first arg


In [2]:
with zipfile.ZipFile('/home/universe/.mxnet/datasets/data/jaychou_lyrics.txt.zip', 'r') as zin:
    zin.extractall('/home/universe/.mxnet/datasets/data/')

with open('/home/universe/.mxnet/datasets/data/jaychou_lyrics.txt') as f:
    corpus_chars = f.read()

corpus_chars[0:50]

'想要有直升机\n想要和你飞到宇宙去\n想要和你融化在一起\n融化在宇宙里\n我每天每天每天在想想想想著你\n这'

In [3]:
len(corpus_chars)

63282

In [4]:
corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
corpus_chars = corpus_chars[0:20000]
corpus_chars[0:50]

'想要有直升机 想要和你飞到宇宙去 想要和你融化在一起 融化在宇宙里 我每天每天每天在想想想想著你 这'

In [5]:
idx_to_char = list(set(corpus_chars))
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
vocab_size = len(char_to_idx)
vocab_size

1447

In [6]:
corpus_indices = [char_to_idx[char] for char in corpus_chars]
sample = corpus_indices[:40]
print('chars: \n', ''.join([idx_to_char[idx] for idx in sample]))
print('\nindices: \n', sample)

chars: 
 想要有直升机 想要和你飞到宇宙去 想要和你融化在一起 融化在宇宙里 我每天每天每

indices: 
 [886, 1045, 746, 103, 109, 1214, 29, 886, 1045, 1176, 431, 1297, 315, 422, 1, 1220, 29, 886, 1045, 1176, 431, 907, 1033, 1203, 558, 953, 29, 907, 1033, 1203, 422, 1, 704, 29, 786, 1254, 522, 1254, 522, 1254]


In [7]:
def data_iter_random(corpus_indices, batch_size, num_steps, ctx=None):
    num_inputs = len(corpus_indices) - 1
    num_examples = num_inputs // num_steps
    epoch_size = num_examples // batch_size
#     print(num_examples, epoch_size)
    
    example_indices = list(range(num_examples))
    random.shuffle(example_indices)
#     print(example_indices)
    def _data(pos):
        return corpus_indices[pos*num_steps: (pos+1)*num_steps]
    for i in range(epoch_size):
        i = i * batch_size
#         print('i=',i)
        batch_indices = example_indices[i: i+batch_size]
        X = nd.array([_data(j) for j in batch_indices], ctx=ctx)
        Y = nd.array([_data(j+1) for j in batch_indices], ctx=ctx)
        yield X, Y
    

In [25]:
def data_iter_consecutive(corpus_indices, batch_size, num_steps, ctx=None):
    corpus_indices = nd.array(corpus_indices, ctx=ctx)
    data_len = len(corpus_indices)
    batch_len = data_len // batch_size
    indices = corpus_indices[0: batch_size*batch_len].reshape((
        batch_size, batch_len))
    # 减一是因为输出的索引是相应输入的索引加一。
    epoch_size = (batch_len - 1) // num_steps
    for i in range(epoch_size):
        i = i * num_steps
        X = indices[:, i: i+num_steps]
        Y = indices[:, i+1: i+num_steps+1]
        yield X, Y

In [11]:
my_seq = list(range(30))
# for X, Y in data_iter_random(my_seq, batch_size=2, num_steps=3):
#     print('X: ', X, '\nY:', Y, '\n')

In [12]:
def to_onehot(X, size):
    return [nd.one_hot(x, size) for x in X.T]

print(X.T)
get_inputs = to_onehot
inputs = get_inputs(X, vocab_size)
len(inputs), inputs[0].shape


[[15. 12.]
 [16. 13.]
 [17. 14.]]
<NDArray 3x2 @cpu(0)>


(3, (2, 1447))

In [13]:
ctx = utils.try_gpu()
print('will use', ctx)

num_inputs = vocab_size
num_hiddens = 256
num_outputs = vocab_size

def get_params():
    # 隐藏层参数。
    W_xh = nd.random.normal(scale=0.01, shape=(num_inputs, num_hiddens),
                            ctx=ctx)
    W_hh = nd.random.normal(scale=0.01, shape=(num_hiddens, num_hiddens),
                            ctx=ctx)
    b_h = nd.zeros(num_hiddens, ctx=ctx)
    # 输出层参数。
    W_hy = nd.random.normal(scale=0.01, shape=(num_hiddens, num_outputs),
                            ctx=ctx)
    b_y = nd.zeros(num_outputs, ctx=ctx)

    params = [W_xh, W_hh, b_h, W_hy, b_y]
    for param in params:
        param.attach_grad()
    return params

will use gpu(0)


In [14]:
def rnn(inputs, state, *params):
    H = state
    W_xh, W_hh, b_h, W_hy, b_y = params
    outputs = []
    for X in inputs:
        H = nd.tanh(nd.dot(X, W_xh) + nd.dot(H, W_hh) + b_h)
        Y = nd.dot(H, W_hy) + b_y
        outputs.append(Y)
    return outputs, H

In [15]:
state = nd.zeros(shape=(X.shape[0], num_hiddens), ctx=ctx)
params = get_params()
outputs, state_new = rnn(get_inputs(X.as_in_context(ctx), vocab_size), state,
                         *params)
len(outputs), outputs[0].shape, state_new.shape

(3, (2, 1447), (2, 256))

In [37]:
# 以下函数预测基于前缀prefix接下来的num_chars个字符
def predict_rnn(rnn, prefix, num_chars, params, num_hiddens, vocab_size, ctx,
               idx_to_char, char_to_idx, get_inputs, is_lstm=False):
    prefix = prefix.lower()
    state_h = nd.zeros(shape=(1, num_hiddens), ctx=ctx)
    if is_lstm:
        state_c = nd.zeros(shape=(1, num_hiddens), ctx=ctx)
    output = [char_to_idx[prefix[0]]]
    for i in range(num_chars + len(prefix)):
        X = nd.array([output[-1]], ctx=ctx)
        if is_lstm:
            Y, state_h, state_c = rnn(getinputs(X, vocab_size), state_h, state_c, *params)
        else:
            Y, state_h = rnn(get_inputs(X, vocab_size), state_h, *params)
        if i < len(prefix) - 1:
            next_input = char_to_idx[prefix[i+1]]
        else: 
            next_input = int(Y[0].argmax(axis=1).asscalar())
        output.append(next_input)
    return ''.join([idx_to_char[i] for i in output])

In [38]:
# def predict_rnn(rnn, prefix, num_chars, params, num_hiddens, vocab_size, ctx,
#                 idx_to_char, char_to_idx, get_inputs, is_lstm=False):
#     prefix = prefix.lower()
#     state_h = nd.zeros(shape=(1, num_hiddens), ctx=ctx)
#     if is_lstm:
#         # 当 RNN 使用 LSTM 时才会用到（后面章节会介绍），本节可以忽略。
#         state_c = nd.zeros(shape=(1, num_hiddens), ctx=ctx)
#     output = [char_to_idx[prefix[0]]]
#     for i in range(num_chars + len(prefix)):
#         X = nd.array([output[-1]], ctx=ctx)
#         # 在序列中循环迭代隐藏状态。
#         if is_lstm:
#             # 当 RNN 使用 LSTM 时才会用到（后面章节会介绍），本节可以忽略。
#             Y, state_h, state_c = rnn(get_inputs(X, vocab_size), state_h,
#                                       state_c, *params)
#         else:
#             Y, state_h = rnn(get_inputs(X, vocab_size), state_h, *params)
#         if i < len(prefix) - 1:
#             next_input = char_to_idx[prefix[i + 1]]
#         else:
#             next_input = int(Y[0].argmax(axis=1).asscalar())
#         output.append(next_input)
#     return ''.join([idx_to_char[i] for i in output])

In [39]:
def grad_clipping(params, state_h, Y, theta, ctx):
    if theta is not None:
        norm = nd.array([0.0], ctx)
        for param in params:
            norm += (param.grad ** 2).sum()
        norm = norm.sqrt().asscalar()
        if norm > theta:
            for param in params:
                param.grad[:] *= theta / norm

In [40]:
def train_and_predict_rnn(rnn, is_random_iter, num_epochs, num_steps,
                          num_hiddens, lr, clipping_theta, batch_size,
                          vocab_size, pred_period, pred_len, prefixes,
                          get_params, get_inputs, ctx, corpus_indices,
                          idx_to_char, char_to_idx, is_lstm=False):
    if is_random_iter:
        data_iter = data_iter_random
    else: 
        data_iter = data_iter_consecutive
    
    params = get_params()
    loss = gloss.SoftmaxCrossEntropyLoss()
    
    for epoch in range(1, num_epochs + 1):
        if not is_random_iter:
            state_h = nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx)
            if is_lstm:
                state_c = nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx)
        train_l_sum = nd.array([0], ctx=ctx)
        train_l_cnt = 0
        for X, Y in data_iter(corpus_indices, batch_size, num_steps, ctx):
            if is_random_iter:
                state_h = nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx)
                if is_lstm:
                    state_c = nd.zeros(shape=(batch_size, num_hiddens),
                                       ctx=ctx)
            else:
                state_h = state_h.detach()
                if is_lstm:
                    state_c = state_c.detach()
            with autograd.record():
                if is_lstm:
                    outputs, state_h, state_c = rnn(
                        get_inputs(X, vocab_size), state_h, state_c, *params)
                else: 
                    outputs, state_h = rnn(
                        get_inputs(X, vocab_size), state_h, *params)
                # 设 t_ib_j 为时间步 i 批量中的元素 j：
                # y 形状：（batch_size * num_steps,）
                # y = [t_0b_0, t_0b_1, ..., t_1b_0, t_1b_1, ..., ]。
                y = Y.T.reshape((-1,))
#                 print (y)
                # 拼接 outputs，形状：(batch_size * num_steps, vocab_size)。
                outputs = nd.concat(*outputs, dim=0)
                l = loss(outputs, y)
            l.backward()
             # 裁剪梯度。
            grad_clipping(params, state_h, Y, clipping_theta, ctx)
            utils.SGD(params, lr)
            train_l_sum = train_l_sum + l.sum()
            train_l_cnt += l.size
        if epoch % pred_period == 0:
            print("\nepoch %d, perplexity %f"
                  % (epoch, (train_l_sum / train_l_cnt).exp().asscalar()))
            for prefix in prefixes:
                print(' - ', predict_rnn(
                    rnn, prefix, pred_len, params, num_hiddens, vocab_size,
                    ctx, idx_to_char, char_to_idx, get_inputs, is_lstm))

In [41]:
num_epochs = 200
num_steps = 35
batch_size = 32
lr = 0.2
clipping_theta = 5
prefixes = ['分开', '不分开']
pred_period = 40
pred_len = 100
train_and_predict_rnn(rnn, False, num_epochs, num_steps, num_hiddens, lr,
                      clipping_theta, batch_size, vocab_size, pred_period,
                      pred_len, prefixes, get_params, get_inputs, ctx,
                      corpus_indices, idx_to_char, char_to_idx)


epoch 40, perplexity 65.488914
 -  分开 我不要再样 我不要再不 我不能再不 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再
 -  不分开 一直用剧 我想了这样  哼有我们留 你一种味道 有一种味道 他不是我们 你不能 不想我 别你的公式我 不知我 别怪我 别你的公式我 不知我 别怪我 别你的公式我 不知我 别怪我 别你的公式我 不知我 

epoch 80, perplexity 9.116345
 -  分开 我想要陪你走着 一身好酒 你面一碗热粥 配上几斤的牛肉 我说店小二 三两银够不够 景色入秋 漫天黄沙截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快使用双截棍
 -  不分开 我的想 放你载睛看着我 别发抖 快给我抬起头 有话去对医药箱一场悲 我不能 想不不到陪我 不散为真 我该好这生活 后知后觉 又使了双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快使

epoch 120, perplexity 3.701919
 -  分开了到 学许这种和著 篮使将那的画 还过着你还得这样 我想就这样  你说不起不是 轻静了我的微知 没说能在去的我 该要这力再的背 我想要你拆远开著我 爱不休 别给我抬起头 有话去对医药箱说 别怪我 别怪我
 -  不分开 我没有这你 我的爱空 被话着外的溪边我 我们不见就整去前 不想再笑 你耀的背我感多的可爱女人 透坏的让我疯狂的可爱女人 透坏的让我疯狂的可爱女人 透坏的让我疯狂的可爱女人 透坏的让我疯狂的可爱女人 透

epoch 160, perplexity 2.601635
 -  分开 我想要再你 我不着我不要 又情的欢火我 你的灵应 你想靠很 乐天风通 过人 在箩  有炭么 太快的手如就 包我的天一来 还室的那一间 我怎么看不见 消失的下雨天 我好想再淋一遍 没想到失去的勇气我还留
 -  不分开  没有你有我说你要没有难样 拜托得大事我们不起很年幼 而如 我想是你的脑袋有问题上 但不到的茶 在一种味道叫做家 他法泡剔它 喝感时觉还不差 陆羽泡的茶 听说名和利都不拿 他牵着一匹瘦马在走天涯 爷爷

epoc

In [ ]:
def train_and_predict_rnn(rnn, is_random_iter, num_epochs, num_steps,
                          num_hiddens, lr, clipping_theta, batch_size,
                          vocab_size, pred_period, pred_len, prefixes,
                          get_params, get_inputs, ctx, corpus_indices,
                          idx_to_char, char_to_idx, is_lstm=False):
    if is_random_iter:
        data_iter = data_iter_random
    else:
        data_iter = data_iter_consecutive
    params = get_params()
    loss = gloss.SoftmaxCrossEntropyLoss()

    for epoch in range(1, num_epochs + 1):
        # 如使用相邻采样，隐藏变量只需在该 epoch 开始时初始化。
        if not is_random_iter:
            state_h = nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx)
            if is_lstm:
                state_c = nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx)
        train_l_sum = nd.array([0], ctx=ctx)
        train_l_cnt = 0
        for X, Y in data_iter(corpus_indices, batch_size, num_steps, ctx):
            # 如使用随机采样，读取每个随机小批量前都需要初始化隐藏变量。
            if is_random_iter:
                state_h = nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx)
                if is_lstm:
                    state_c = nd.zeros(shape=(batch_size, num_hiddens),
                                       ctx=ctx)
            # 如使用相邻采样，需要使用 detach 函数从计算图分离隐藏状态变量。
            else:
                state_h = state_h.detach()
                if is_lstm:
                    state_c = state_c.detach()
            with autograd.record():
                # outputs 形状：(batch_size, vocab_size)。
                if is_lstm:
                    outputs, state_h, state_c = rnn(
                        get_inputs(X, vocab_size), state_h, state_c, *params)
                else:
                    outputs, state_h = rnn(
                        get_inputs(X, vocab_size), state_h, *params)
                # 设 t_ib_j 为时间步 i 批量中的元素 j：
                # y 形状：（batch_size * num_steps,）
                # y = [t_0b_0, t_0b_1, ..., t_1b_0, t_1b_1, ..., ]。
                y = Y.T.reshape((-1,))
                # 拼接 outputs，形状：(batch_size * num_steps, vocab_size)。
                outputs = nd.concat(*outputs, dim=0)
                l = loss(outputs, y)
            l.backward()
            # 裁剪梯度。
            grad_clipping(params, state_h, Y, clipping_theta, ctx)
            gb.sgd(params, lr, 1)
            train_l_sum = train_l_sum + l.sum()
            train_l_cnt += l.size
        if epoch % pred_period == 0:
            print("\nepoch %d, perplexity %f"
                  % (epoch, (train_l_sum / train_l_cnt).exp().asscalar()))
            for prefix in prefixes:
                print(' - ', predict_rnn(
                    rnn, prefix, pred_len, params, num_hiddens, vocab_size,
                    ctx, idx_to_char, char_to_idx, get_inputs, is_lstm))